In [1]:
#using Pkg
#Pkg.add("JSON")
#Pkg.add("DotEnv")
#Pkg.add("HTTP")
#Pkg.add("JSON")
#Pkg.add("URIs")
#Pkg.add("ConfigEnv")

using DotEnv, HTTP, URIs,JSON


In [10]:
"""
    get_data(property)

This function requests data from an api using an access_token.  The variable property is a string.  Each property
gives access to a different set of data.  The set of admissable properties is described in accompanying documentation.

Since access tokens
are fetched in many different ways, this function does not fetch tokens.  Instead it relies
on an outside program fetching the token and saving it in a file called `.env`.

The `.env` file has to contain at least two fields which look like
```
api_url=https://yoursite.ca/api
access_token=yourtoken
```
Note that there is no trailing slash on the api url.  The program appends the variable property to the end of the api_url when fetching.

The program should return either an array of json strings, or a single json string with an error message in it. 

"""
function get_data(property) 
    cfg = DotEnv.config()
    #a bogus token which works for public request
    default_token="eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJpZCI6IjhjNzRhMjY0YjE0Yj"
    if isempty(cfg)
        response = Dict(
            "error" => "configuration",
            "error_description" => ".env files doesn't exist or is empty"
            )
        return JSON.json(response)
    elseif cfg["api_url"] == ""
        response = Dict(
            "error" => "api_url",
            "error_description" => "there is no api_url, can't return output"
            )
        return JSON.json(response)
    elseif cfg["access_token"] == ""
        #assuming this is a public request
        cfg["access_token"] = default_token
    end
                
    api_url = string(cfg["api_url"],"/",property)
    new_headers = Dict(
        "Accept" => "application/json",
        "Authorization" => string("Bearer ",cfg["access_token"]),
    )
    response = HTTP.request("GET", api_url, new_headers)
        
    r = String(response.body)
    check = JSON.parse(r)
    if "error" in keys(check)
        println("detected error")
    end
    return r
end

get_data

In [11]:

   println( get_data("gendered_data"))
    


detected error
{"error":"oauth2 error","error_description":"Expired token"}
